In [ ]:
import beamtest_analysis_helper as helper
from glob import glob
from natsort import natsorted
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np

# import hist
# import matplotlib.pyplot as plt
# import matplotlib.ticker as ticker
# import matplotlib.colors as colors
# import mplhep as hep
# hep.style.use('CMS')

In [ ]:
columns_to_read = ['evt', 'board', 'row', 'col', 'toa', 'tot', 'cal']
names = ["ET2_EPIR_Pair1", "ET2p01_BAR_4", "ET2p01_BAR_5", "ET2_EPIR_Pair4"]

high_voltages = [250, 260, 210, 260]
offsets = [15, 10, 10, 15]

chip_figtitles = [
    f"(Trigger) EPIR Pair1 HV{high_voltages[0]}V OS:{offsets[0]}",
    f"(DUT1) Barcelona 4 HV{high_voltages[1]}V OS:{offsets[1]}",
    f"(Reference) Barcelona 5 HV{high_voltages[2]}V OS:{offsets[2]}",
    f"(DUT2) EPIR Pair4 HV{high_voltages[3]}V OS:{offsets[3]}"
]

dut1_id = 1
dut2_id = 3
ref_id = 2

In [ ]:
files = natsorted(glob('DESYFeb2024_Run_34_feather/*feather'))
files[:3]

### Load dataframe based on single pixel selection from the trigger board

In [ ]:
last_evt = 0
dataframes = []

for idx, ifile in enumerate(tqdm(files)):

    tmp_df = pd.read_feather(ifile, columns=columns_to_read)

    if tmp_df.empty:
        continue

    subset_df = tmp_df.loc[tmp_df['evt'].isin(tmp_df.loc[(tmp_df['board'] == 0) & (tmp_df['row'] == 7) & (tmp_df['col'] == 7)]['evt'].unique())]
    del tmp_df

    if subset_df.empty:
        continue

    subset_df.reset_index(drop=True, inplace=True)
    subset_df['evt'] = subset_df.groupby('evt').ngroup().astype('uint64')

    if idx > 0:
        subset_df['evt'] += np.uint64(last_evt)

    last_evt += np.uint64(subset_df['evt'].nunique())

    dataframes.append(subset_df)
    del subset_df

df = pd.concat(dataframes)
df.reset_index(inplace=True, drop=True)
del dataframes

### Basic TDC plots

In [ ]:
h_inclusive = helper.return_hist(df, chipNames=names, chipLabels=[0,1,2,3], hist_bins=[100, 128, 128])

for iboard in [0,1,2,3]:
    helper.plot_1d_TDC_histograms(h_inclusive, names[iboard], names[iboard], chip_figtitles[iboard], save=False,
                                tag="inclusive", fig_tag=", inclusive", slide_friendly=True)

del h_inclusive

### Occupancy map

In [ ]:
helper.plot_occupany_map(df, chipLabels=[0,1,2,3], chipNames=names, fig_title=chip_figtitles, fig_tag='inclusive')

### Filtering by TDC

In [ ]:
## Selecting good hits
tdc_cuts = {}
for idx in [0,1,2,3]:
    # board ID: [CAL LB, CAL UB, TOA LB, TOA UB, TOT LB, TOT UB]
    if idx == 0:
        tdc_cuts[idx] = [df.loc[df['board'] == idx]['cal'].mode()[0]-3, df.loc[df['board'] == idx]['cal'].mode()[0]+3,  100, 500, 100, 250]
    elif idx == ref_id:
        tdc_cuts[idx] = [df.loc[df['board'] == idx]['cal'].mode()[0]-3, df.loc[df['board'] == idx]['cal'].mode()[0]+3,  0, 1100, 80, 200]
    else:
        tdc_cuts[idx] = [df.loc[df['board'] == idx]['cal'].mode()[0]-3, df.loc[df['board'] == idx]['cal'].mode()[0]+3,  0, 1100, 0, 600]

filtered_df = helper.tdc_event_selection(df, tdc_cuts_dict=tdc_cuts, select_by_hit=False)

### Efficiency

In [ ]:
event_board_counts = filtered_df.groupby(['evt', 'board']).size().unstack(fill_value=0)
event_selection_col = None

trig_selection = (event_board_counts[0] == 1)
ref_selection = (event_board_counts[ref_id] == 1)
event_selection_col = trig_selection & ref_selection

case1_df = filtered_df[filtered_df['evt'].isin(event_board_counts[event_selection_col].index)]
case1_df.reset_index(inplace=True, drop=True)

In [ ]:
diff_row = abs(case1_df[case1_df['board'] == 0]['row'].values - case1_df[case1_df['board'] == ref_id]['row'].values)
diff_col = abs(case1_df[case1_df['board'] == 0]['col'].values - case1_df[case1_df['board'] == ref_id]['col'].values)

In [ ]:
## Select event numbers where trig - ref colinearity has been satisfied
trig_ref_colinear_evts = case1_df['evt'].unique()[(diff_row <= 1) & (diff_col <= 1)]

In [ ]:
dut1_df = helper.return_broadcast_dataframe(input_df=filtered_df, trig_board_id=0, ref_board_id=ref_id, dut_board_id=dut1_id, trig_dut=True, ref_dut=False)

In [ ]:
colinear_dut1_df = dut1_df[dut1_df['evt'].isin(trig_ref_colinear_evts)].reset_index(drop=True)

In [ ]:
### Board-level Efficiency of DUT w/ considering colinear of trigger and reference boards
# 100*colinear_dut1_df['evt'].nunique()/case1_df['evt'].nunique()
100*colinear_dut1_df['evt'].nunique()/trig_ref_colinear_evts.size

In [ ]:
diff_row = colinear_dut1_df[colinear_dut1_df['board'] == 0]['row'].values - colinear_dut1_df[colinear_dut1_df['board'] == dut1_id]['row'].values
diff_col = colinear_dut1_df[colinear_dut1_df['board'] == 0]['col'].values - colinear_dut1_df[colinear_dut1_df['board'] == dut1_id]['col'].values

In [ ]:
## Select event numbers where trig - dut colinearity has been satisfied
trig_dut_colinear_df = colinear_dut1_df[colinear_dut1_df['board'] == 0][(diff_row <= 2) & (diff_col <= 2)]
trig_dut_colinear_evts = trig_dut_colinear_df['evt'].unique()
print(trig_dut_colinear_evts.size,colinear_dut1_df['evt'].nunique(),trig_ref_colinear_evts.size,dut1_df['evt'].nunique(),case1_df['evt'].nunique())

In [ ]:
# 100*trig_dut_colinear_df['evt'].nunique()/case1_df['evt'].nunique()
100*trig_dut_colinear_df['evt'].nunique()/trig_ref_colinear_evts.size

In [ ]:
colinear_case1_df = case1_df[case1_df['evt'].isin(trig_ref_colinear_evts)].reset_index(drop=True)
hits_count_by_col_row_board = colinear_case1_df.groupby(['col', 'row', 'board'])['evt'].count().reset_index()
# hits_count_by_col_row_board = colinear_dut1_df.groupby(['col', 'row', 'board'])['evt'].count().reset_index()

hits_count_by_col_row_board = hits_count_by_col_row_board.rename(columns={'evt': 'hits'})
denominator = hits_count_by_col_row_board[hits_count_by_col_row_board['board'] == 0].pivot_table(
    index='row',
    columns='col',
    values='hits',
    fill_value=0  # Fill missing values with 0 (if any)
)

In [ ]:
# hits_count_by_col_row_board = trig_dut_colinear_df.groupby(['col', 'row'])['evt'].count().reset_index()
colinear_case2_df = case1_df[case1_df['evt'].isin(trig_dut_colinear_evts)].reset_index(drop=True)
hits_count_by_col_row_board = colinear_case2_df.groupby(['col', 'row', 'board'])['evt'].count().reset_index()
hits_count_by_col_row_board = hits_count_by_col_row_board.rename(columns={'evt': 'hits'})
numerator = hits_count_by_col_row_board[hits_count_by_col_row_board['board'] == 0].pivot_table(
    index='row',
    columns='col',
    values='hits',
    fill_value=0  # Fill missing values with 0 (if any)
)

In [ ]:
eff_table = 100*numerator/denominator
eff_table = eff_table.fillna(0)

# Create a heatmap to visualize the count of hits
fig, ax = plt.subplots(dpi=100, figsize=(20, 20))
im = ax.imshow(eff_table, cmap="viridis", interpolation="nearest")

# Add color bar
cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
cbar.set_label('Efficiency', fontsize=20)
cbar.ax.tick_params(labelsize=20)

for i in range(16):
    for j in range(16):
        try:
            value = eff_table.iloc[i, j]
        except:
            value = -1
        if value == -1: continue
        text_color = 'black' if value > 0.5*(eff_table.values.max() + eff_table.values.min()) else 'white'
        text = str("{:.1f}%".format(value))
        plt.text(j, i, text, va='center', ha='center', color=text_color, fontsize=17)

hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
ax.set_xlabel('Column (col)', fontsize=20)
ax.set_ylabel('Row (row)', fontsize=20)
ticks = range(0, 16)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.set_title(f"Efficiency of DUT projected to the trigger board based on 3-board analysis", loc="right", size=20)
ax.tick_params(axis='x', which='both', length=5, labelsize=17)
ax.tick_params(axis='y', which='both', length=5, labelsize=17)
ax.invert_xaxis()
ax.invert_yaxis()
plt.minorticks_off()

# fig.savefig('dut1_3board_eff_map.png')
# fig.savefig('dut1_3board_eff_map.pdf')